<a href="https://colab.research.google.com/github/hilanistel/STCP_feature_selection/blob/GTCP/GTCP_small_scale_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
TR = {(0,0,0,1):1, (0,1,1,1):1,(1,1,1,1):1,(1,0,2,1):1,(1,0,1,1):0,(0,1,0,0):0,(0,0,2,0):0}

In [113]:
ci = [4,3,6,5]

In [114]:
N = {0:"Negative", 1:"Positive"}

**CRA**

𝑆= {𝑖∈𝑁: 𝑟𝑖≠𝑞𝑖}

In [115]:
def S(r,q):
  s = []
  for index_i,i in enumerate(r):
      if r[index_i] != q[index_i]:
        s.append(index_i)
  return s

In [116]:
S((1, 1, 1, 1),(1, 0, 1, 1))

[1]

If there is no 𝑆′∈ℂ s.t 𝑆′⊆𝑆

In [117]:
# function create to get all N subsets
def sub_lists (l): 
    base = []   
    lists = [base] 
    for i in range(len(l)): 
        orig = lists[:] 
        new = l[i] 
        for j in range(len(lists)): 
            lists[j] = lists[j] + [new] 
        lists = orig + lists   
    lists.remove([])  
    return lists

In [118]:
def S_in_C(S,C):
  subsets_S = sub_lists(S)
  for s in subsets_S:
    for c in C:
      if s == c:
        return 0 #True
  return 1 #False

In [135]:
C = [] # start with an empty collection of constraints
for i in TR:
  for j in TR:
    if TR[i] != TR[j]:
      s = S(i,j) # 𝑆= {𝑖∈𝑁: 𝑟𝑖≠𝑞𝑖}
      if S_in_C(s,C) == 1:
        for c in C:
          C1= C.copy()
          subsets_c = sub_lists(c)
          if s in subsets_c:
            C1.remove(c)
          C = C1
        C.append(s)
print(C)

[[1], [2], [0, 3]]


**CPLEX**

In [148]:
pip install cplex

     |████████████████████████████████| 31.0MB 147kB/s 


In [153]:

import cplex

# Create an instance of a linear problem to solve
problem = cplex.Cplex()


# We want to find a minimum of our objective function
problem.objective.set_sense(problem.objective.sense.minimize)

# The names of our variables (tests)
names = ["x1", "x2", "x3", "x4"]

# The obective function. More precisely, the coefficients of the objective
# function. Note that we are casting to floats.
objective = [4.0,3.0,6.0,5.0]

# Lower bounds. Since these are all zero, we could simply not pass them in as
# all zeroes is the default.
lower_bounds = [0.0, 0.0, 0.0, 0.0]

# Upper bounds. The default here would be cplex.infinity, or 1e+20.
upper_bounds = [100, 1000, 1000, 1000]

problem.variables.add(obj = objective,
                      lb = lower_bounds,
                      ub = upper_bounds,
                      names = names)

# Constraints
constraint_names = ["c1", "c2", "c3"]

first_constraint = [["x2"], [1.0]]
second_constraint = [["x3"], [1.0]]
third_constraint = [["x1", "x4"], [1.0,1.0]]

constraints = [ first_constraint, second_constraint, third_constraint ]

rhs = [1.0, 1.0, 1.0]
constraint_senses = [ "G", "G" , "G"]

# And add the constraints
problem.linear_constraints.add(lin_expr = constraints,
                               senses = constraint_senses,
                               rhs = rhs,
                               names = constraint_names)

# Solve the problem
problem.solve()

# And print the solutions
print(problem.solution.get_values())

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 2 rows and 2 columns.
Reduced LP has 1 rows, 2 columns, and 2 nonzeros.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =            13.000000
[1.0, 1.0, 1.0, 0.0]
